# PIK Digital Day

### Utilities

In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from tqdm import tqdm, tqdm_notebook

In [2]:
def rmse(a, b):
    return mean_squared_error(a, b)**0.5

### Load Data

In [3]:
train = pd.read_csv("train.csv", encoding="cp1251")
test = pd.read_csv("test.csv", encoding="cp1251")
test_id = test["id"]

train = train.drop(['vid_2', 'plan_s', 'plan_l', 'vid_0', 'plan_m', 'start_square', 'vid_1'], axis=1)

train["is_train"] = True
test["is_train"] = False

data = pd.concat((train, test))
data = data.drop("date1", axis=1)

/home/alxmamaev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Remove the CWD from sys.path while we load stuff.


### Preprocess Data

In [4]:
data.head(3)

,Cтавка по ипотеке,bulk_id,id,is_train,mean_fl,mean_sq,month,month_cnt,price,spalen,...,Площадь зеленой зоны в радиусе 500 м,Площадь земельного участка,Площадь пром. зоны в радиусе 500 м,Подземная парковка,Поликлиника,Система мусоротведения,Спортивная площадка,Станций метро от кольца,ФОК,Школа
0,10.04,FF3814A9-9F7D-E711-8530-00505688958B,0,True,8,38.00,10,34,135633.913043,1,...,25533,17.0,65357,да,0,контейнеры,да,8,0,825
1,10.04,FF3814A9-9F7D-E711-8530-00505688958B,1,True,9,58.01,10,34,128492.419825,2,...,25533,17.0,65357,да,0,контейнеры,да,8,0,825
2,10.04,FF3814A9-9F7D-E711-8530-00505688958B,2,True,10,22.58,10,34,158237.634409,0,...,25533,17.0,65357,да,0,контейнеры,да,8,0,825


In [5]:
categorical_features = []
for i in data.drop("bulk_id", axis=1).columns:
    if data[i].dtype =="object":
        categorical_features.append(i)

categorical_features

['Автомойка',
 'Входные группы',
 'Двор без машин',
 'Кладовые',
 'Класс объекта',
 'Колясочные',
 'Огорожена территория',
 'Подземная парковка',
 'Система мусоротведения',
 'Спортивная площадка']

In [6]:
for cat in categorical_features:
    data[cat] = pd.factorize(data[cat])[0]

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11061 entries, 0 to 1816
Data columns (total 49 columns):
Cтавка по ипотеке                        11061 non-null float64
bulk_id                                  11061 non-null object
id                                       11061 non-null int64
is_train                                 11061 non-null bool
mean_fl                                  11061 non-null int64
mean_sq                                  11061 non-null float64
month                                    11061 non-null int64
month_cnt                                11061 non-null int64
price                                    11061 non-null float64
spalen                                   11061 non-null int64
value                                    9244 non-null float64
Автомойка                                11061 non-null int64
Вентлияция                               11061 non-null int64
Видеонаблюдение                          11061 non-null int64
Вклады до 1 года 

In [8]:
flat = pd.read_csv("flat.csv", encoding="cp1251")

Множество за период по корпусу/комнатности (далее Сет)- подможество из Множества за период по проекту  состоящее из квартир с одинаковой комнатностью(spalen) и из одного корпуса (bulk). Предсказание делается для каждого такого множества.

In [9]:
flat_real_features = ["floor", "balcon", "square", "Количество помещений"]
flat_cat_features = ["vid", "otdelka", "Класс объекта",
                    "Огорожена территория", "Кондиционирование", "Вентлияция",
                    "Система мусоротведения"]

for bulk_id in tqdm_notebook(data["bulk_id"].unique()):
    set_spalen = data[data["bulk_id"] == bulk_id]["spalen"].unique()
    for spalen in set_spalen:
        data_indexes = (data["bulk_id"] == bulk_id) & (data["spalen"] == spalen)
        flat_indexes = (flat["bulk_id"] == bulk_id) & (flat["spalen"] == spalen)
        
        for f in flat_real_features:
            data.loc[data_indexes, "max_"+f] = flat[flat_indexes][f].max()
            data.loc[data_indexes, "min_"+f] = flat[flat_indexes][f].min()
            
            data.loc[data_indexes, "mean_"+f] = flat[flat_indexes][f].mean()
            data.loc[data_indexes, "median_"+f] = flat[flat_indexes][f].median()
            
            data.loc[data_indexes, "variance_"+f] = ((flat[flat_indexes][f] - flat[flat_indexes][f].mean())**2).mean()
            data.loc[data_indexes, "std_"+f] = ((flat[flat_indexes][f] \
                                                 - flat[flat_indexes][f].mean())**2).mean() ** 0.5
        for f in flat_cat_features:
            for c in flat[f].unique():
                flat_indexes2 = (flat_indexes & (flat[f] == c))
                data.loc[data_indexes, f+"_"+str(c)+"_count"] = flat_indexes2.sum()
                data.loc[data_indexes, f+"_"+str(c)+"_share_object"] = flat_indexes2.sum()/flat_indexes.sum()

In [10]:
data["Средняя цена за квадрат"] = data["price"]/data["mean_sq"]
data["Средняя цена за комнату"] = data["price"]/(data["spalen"]+1)
data["Средняя площадь комнаты"] = data["mean_sq"]/(data["spalen"]+1)
data["Средняя площадь комнаты"] = data["mean_sq"]/(data["spalen"]+1)

data["Удобство до большой дороги"] = data["До большой дороги на машине(км)"]/data["Машиномест"]
data["Удобство до авторазвязки"] = data["До удобной авторазвязки на машине(км)"]/data["Машиномест"]
data["Удобство до парка на машине"] = data["До парка(км)"]/data["Машиномест"]

data["Удобство до кремля пешком"] = data["До Кремля"]/data["До метро пешком(км)"]
data["Удобство до парка пешком"] = data["До парка(км)"]/data["До метро пешком(км)"]
data["Критерий удобства Метро"] = data["Станций метро от кольца"]/data["До метро пешком(км)"]
data["price_USD"] = data["price"]/data["Курс"]

data["Общая сумма вкладов"] = data["Вклады от 1 года до 3 лет"] + data['Вклады свыше 3 лет'] + data['Вклады до 1 года']
data["Соотношение вкладов до 1 и после 1"] = data['Вклады до 1 года'] / (data["Вклады от 1 года до 3 лет"] + data['Вклады свыше 3 лет'])
data["Соотношение вкладов до 3 и после 3"] = (data['Вклады до 1 года'] + data["Вклады от 1 года до 3 лет"]) / data['Вклады свыше 3 лет']

In [11]:
data = data.drop(["id", "bulk_id"], axis=1)
data = data.fillna(-1000)
train = data[data["is_train"] == True]
test = data[data["is_train"] == False]

### Train model

In [12]:
n_folds = 10
max_depth = 10
iterations = 3200

In [13]:
kf = KFold(n_splits=n_folds, random_state=42, shuffle=True)

In [14]:
for i, (train_index, valid_index) in enumerate(kf.split(range(len(train)))):
    train_fold = train.loc[train_index]
    valid_fold = train.loc[valid_index]

    train_fold.to_csv(f"./input2/train_{i}.csv",index=False)
    valid_fold.to_csv(f"./input2/valid_{i}.csv",index=False)

In [22]:
from sklearn.linear_model import ElasticNet, SGDRegressor
from sklearn.preprocessing import MinMaxScaler

In [ ]:
mms.fit_transform

In [98]:
models_predict = []
feature_importances = []
models_rmse = []

for i in tqdm(range(n_folds)):
    train_fold = pd.read_csv(f"./input2/train_{i}.csv")
    valid_fold = pd.read_csv(f"./input2/valid_{i}.csv")
    
    mms = MinMaxScaler()
    
    x_train, y_train = train_fold.drop("value", axis=1), train_fold["value"]
    x_val, y_val = valid_fold.drop("value", axis=1), valid_fold["value"]
    x_test = test.drop("value", axis=1)
    
    x_train = mms.fit_transform(x_train)
    x_val = mms.transform(x_val)
    x_test = mms.transform(x_test)
    
    model = SGDRegressor(eta0=0.0514)
    
    model.fit(x_train, y_train)#, cat_features=cat_cols_indexes)
    
    models_rmse.append(rmse(model.predict(x_val), y_val))
    pred = model.predict(x_test)
    models_predict.append(pred)
    feature_importances.append(model.coef_)

  0%|          | 0/10 [00:00<?, ?it/s]/home/alxmamaev/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
100%|██████████| 10/10 [00:05<00:00,  1.92it/s]


In [99]:
print("Mean RMSE:", np.mean(models_rmse))

Mean RMSE: 298.60839115286024


In [100]:
feature_importances = np.mean(feature_importances[1:], axis=0)
columns = train.drop("value", axis=1).columns

sorted(list(zip(feature_importances, columns)))[::-1]

[(323.0027954679685, 'vid_хороший_count'),
 (256.5983966841725, 'otdelka_Стандартная эконом._share_object'),
 (250.26902391171518, 'Класс объекта_комфорт_count'),
 (197.03246358324748, 'otdelka_ПИК1-лоукост_count'),
 (179.6767521015662, 'otdelka_ПИК2-лоукост_count'),
 (178.05458760590457, 'Система мусоротведения_контейнеры_count'),
 (170.97645948913387, 'Огорожена территория_нет_count'),
 (154.19585791133065, 'Кондиционирование_1_count'),
 (149.51589109477783, 'Вентлияция_0_count'),
 (143.9008835442328, 'otdelka_Стандартная чистовая_count'),
 (143.66685505416746, 'otdelka_Монолит-лоукост_count'),
 (135.2894060003312, 'max_floor'),
 (131.84660338687593, 'std_square'),
 (127.9064851431024, 'otdelka_Лоукост (без кондиционера)_share_object'),
 (122.59394998045731, 'До парка(км)'),
 (118.46830256372976, 'std_balcon'),
 (118.34333888940176, 'otdelka_ПИК2-лоукост_share_object'),
 (112.20472630153115, 'otdelka_Стандартная эконом._count'),
 (108.32178124261, 'otdelka_Лоукост (без кондиционера)_

In [101]:
predict = np.mean(models_predict, axis=0)
predict[predict<0] = 0
test["value"] = predict

/home/alxmamaev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [102]:
submit = pd.DataFrame()
submit["id"] = test_id
submit["value"] = test["value"]
submit.to_csv("lin-submit.csv", index=False)